In [122]:
%%time
# Program to pickle historical data for SD of nse underlyings

# STATUS: Completed
# Runtime: 12 mins (for a new dataset)

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

import pandas as pd
import numpy as np
import datetime
import os

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'

Wall time: 137 ms


In [191]:
#......     Get the fills    ........
f = ib.fills()
ib.sleep(0.5)
f

df_fills = pd.DataFrame([[i.contract.localSymbol, i.contract.conId, i.execution.cumQty, 
  i.execution.avgPrice, i.commissionReport.commission, i.contract] for i in f], 
  columns = ['localSymbol', 'conId', 'cumQty', 'avgPrice', 'commission', 'contract'])

#......     Get the positions   .......
p = ib.positions()

df_positions = pd.DataFrame([(i.contract.conId, i.position, i.avgCost) for i in p], 
             columns=['conId', 'position', 'avgCost'])

df_positions1 = df_positions.merge(df_fills, on='conId',
                                   how='left').drop_duplicates('conId').reset_index(drop=True)

#......     Get the market price ........
mkt_data = [ib.reqMktData(df_positions1.contract[c]) for c in range(len(df_positions1))]
ib.sleep(5)
mkt_data

df_mktprice = pd.DataFrame([(t.contract.conId, t.bid, t.ask, t.last) 
              for t in mkt_data], 
             columns=['conId', 'bid', 'ask', 'last'])

df_positions2 = df_positions1.merge(df_mktprice, on='conId', 
                    how='left').drop_duplicates('conId').reset_index(drop=True)

df_positions2['Gain'] = (df_positions2.avgCost-df_positions2['last'])/df_positions2.avgCost

df_positions2['avgBidLast'] = df_positions2[['bid', 'last']].mean(axis=1)
df_positions2['tgt_price'] = df_positions2[['avgBidLast', 'avgCost']].min(axis=1)

cols = ['localSymbol', 'position', 'Gain', 'avgCost', 'bid', 'ask', 
        'last', 'tgt_price', 'avgBidLast', 'contract']
df_positions3 = df_positions2[cols].sort_values('Gain', ascending=False)

Started to throttle requests
Stopped to throttle requests


In [194]:
ib.reqHistoricalData(contract=df_positions3.contract[0], endDateTime='', 
                                    durationStr='1 D', barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)

[BarData(date=datetime.date(2018, 11, 30), open=0.75, high=0.75, low=0.7, close=0.7, volume=4000, barCount=2, average=0.725)]

In [195]:
df_positions3

,localSymbol,position,Gain,avgCost,bid,ask,last,tgt_price,avgBidLast,contract
127,ASIANPAINT18DEC1100PE,-600.0,0.305985,0.360223,-1.0,-1.0,0.25,-0.375000,-0.375,"Option(conId=335380246, symbol='ASIANPAIN', la..."
76,SUNPHARMA18DEC420PE,-1100.0,0.297685,3.274882,-1.0,-1.0,2.30,0.650000,0.650,"Option(conId=335613836, symbol='SUNPHARMA', la..."
100,ICICIPRULI18DEC285PE,-1300.0,0.247038,0.531236,-1.0,-1.0,0.40,-0.300000,-0.300,"Option(conId=335398830, symbol='ICICIPRUL', la..."
146,BHEL18DEC60PE,-7500.0,0.226164,0.646132,-1.0,-1.0,0.50,-0.250000,-0.250,"Option(conId=335418671, symbol='BHEL', lastTra..."
67,NIFTY18DEC8200PE,-75.0,0.221098,2.182558,-1.0,-1.0,1.70,0.350000,0.350,"Option(conId=141093844, symbol='NIFTY50', last..."
95,NBCC18DEC42.5PE,-6000.0,0.154645,0.295734,-1.0,-1.0,0.25,-0.375000,-0.375,"Option(conId=335410103, symbol='NBCC', lastTra..."
36,NIFTY18DEC9600PE,-75.0,0.152781,9.324622,-1.0,-1.0,7.90,3.450000,3.450,"Option(conId=151660781, symbol='NIFTY50', last..."
136,HINDALCO18DEC190PE,-3500.0,0.137747,1.391703,-1.0,-1.0,1.20,0.100000,0.100,"Option(conId=335422781, symbol='HINDALCO', las..."
81,NIFTY18DEC9900PE,-75.0,0.133676,20.662023,-1.0,-1.0,17.90,8.450000,8.450,"Option(conId=154450268, symbol='NIFTY50', last..."
162,TATASTEEL18DEC420PE,-1061.0,0.129675,0.976647,-1.0,-1.0,0.85,-0.075000,-0.075,"Option(conId=335433038, symbol='TATASTEEL', la..."
